# This notebook is used to process the gantry data and save it as a .pt file

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
import csv
from pathlib import Path

## load data

In [ ]:
## Params
NUM_POS = 4096
DARK_THRESHOLD = None

# load data from csv file
raw_data = np.loadtxt('gantry_data_1.csv', delimiter=',',usecols=(2,3,4,5))
data_valid = np.array([_ for _ in raw_data if _ > DARK_THRESHOLD])

In [17]:
# caluculate resolution in 18 meters, beam moving resolution
import math
math.tan((1/3517)*280e-6)*18e3

0.001433039522320162

In [ ]:
# plot the data
plt.figure(figsize=(10,10))
for i in range(4):
    plt.scatter(range(len(data_valid)),data_valid[:,i],label=f'diod_{i+1}')
plt.legend()
plt.show()

## Form train and valida data set

In [27]:
# Form train set
#### at each position in ((-8,8)step size 0.25, total 64*64 positions), random sampling 3 data points
## form train input set
train_input = data_valid[]
## form train target set
# make a list of number from -8 to 8 at step size 0.25, and each of the number repeat 3 times
train_target = np.repeat(np.arange(-8,8,0.25),3)

In [30]:
# Form valida set
####at each position in ((-8,8)step size 0.25, total 64*64 positions), random sampling 1 data point
val_input = np.zeros((64,64,4))
val_target = np.arange(-8,8,0.25)

In [28]:
train_target

array([-8.  , -8.  , -8.  , -7.75, -7.75, -7.75, -7.5 , -7.5 , -7.5 ,
       -7.25, -7.25, -7.25, -7.  , -7.  , -7.  , -6.75, -6.75, -6.75,
       -6.5 , -6.5 , -6.5 , -6.25, -6.25, -6.25, -6.  , -6.  , -6.  ,
       -5.75, -5.75, -5.75, -5.5 , -5.5 , -5.5 , -5.25, -5.25, -5.25,
       -5.  , -5.  , -5.  , -4.75, -4.75, -4.75, -4.5 , -4.5 , -4.5 ,
       -4.25, -4.25, -4.25, -4.  , -4.  , -4.  , -3.75, -3.75, -3.75,
       -3.5 , -3.5 , -3.5 , -3.25, -3.25, -3.25, -3.  , -3.  , -3.  ,
       -2.75, -2.75, -2.75, -2.5 , -2.5 , -2.5 , -2.25, -2.25, -2.25,
       -2.  , -2.  , -2.  , -1.75, -1.75, -1.75, -1.5 , -1.5 , -1.5 ,
       -1.25, -1.25, -1.25, -1.  , -1.  , -1.  , -0.75, -0.75, -0.75,
       -0.5 , -0.5 , -0.5 , -0.25, -0.25, -0.25,  0.  ,  0.  ,  0.  ,
        0.25,  0.25,  0.25,  0.5 ,  0.5 ,  0.5 ,  0.75,  0.75,  0.75,
        1.  ,  1.  ,  1.  ,  1.25,  1.25,  1.25,  1.5 ,  1.5 ,  1.5 ,
        1.75,  1.75,  1.75,  2.  ,  2.  ,  2.  ,  2.25,  2.25,  2.25,
        2.5 ,  2.5 ,

# Real gantry data processing

## load data and visualize

fix the raw file and aplit line not enough data issue

In [23]:
# Define paths
data_root_path = Path.cwd().parent/'data'
raw_data_path = data_root_path/'pd_data'/'gantry_0213_160012.csv'

# Read and process the file
lines = raw_data_path.read_text().splitlines()[1:]  # skip the first headerline
modified_lines = []

for line in lines:
    line_list = line.split(',')
    if line_list[1].startswith('s') or line_list[1].startswith('W'):
        line = f"{line.strip()},0.0,0.0,0.0,0.0"
    modified_lines.append(line)

# Write back to file
raw_data_path.write_text('\n'.join(modified_lines))
# raw_data = np.loadtxt(raw_data_path, delimiter=',',skiprows=1,usecols=range(2,10))


1872498

## SORT OUT INPUT DATA

In [74]:
data_root_path = Path.cwd().parent/'data'

pd_data_path = data_root_path/'pd_data'
gantry_data_path = data_root_path/'gantry_pos_data'

pd_data = []
for file in Path(pd_data_path).glob('gantry*.csv'):
    # go through all the line in file, if the 
    # split the line by comma and convert the string to int
    line_data = []
    for line in file.read_text().split('\n'):
        line = line.split(',')
        if line[1] in ['s','W']:
            continue
        line_data.extend([int(float(x)) for x in line[2:]])
        if line[1] == 'G4':
            pd_data.append(line_data)
            line_data = []
    print(len(pd_data)) 


6603
13192


In [73]:
len(pd_data[1])

32

## SORT OUT TARGET DATA

In [27]:
data_root_path = Path.cwd().parent/'data'

pd_data_path = data_root_path/'pd_data'

# create target set
tar_data = np.array([[x,y] for y in np.arange(-8,8.5,0.5) for x in np.arange(-8,8.5,0.5)])

# sort out input set
pd_data = []

for file in Path(pd_data_path).glob('gantry*.csv'):
    position_idx = -1
    # go through all the line in file, if the 
    # split the line by comma and convert the string to int
    line_data = []
    for line in file.read_text().split('\n'):
        line = line.split(',')

        if line[1] in ['s','W']:
            if line[1] == 's':
                position_idx += 1
            continue

        line_data.extend([int(float(x)) for x in line[2:]])
        if line[1] == 'G4':
            # extend the line with tar_data
            line_data.extend(tar_data[position_idx])
            # add the line to input data set(pd_data)
            pd_data.append(line_data)
            line_data = []
pd_data = np.array(pd_data)


In [28]:
print(len(tar_data))

1089


In [31]:
for i in tar_data:
    print(i)

[-8. -8.]
[-7.5 -8. ]
[-7. -8.]
[-6.5 -8. ]
[-6. -8.]
[-5.5 -8. ]
[-5. -8.]
[-4.5 -8. ]
[-4. -8.]
[-3.5 -8. ]
[-3. -8.]
[-2.5 -8. ]
[-2. -8.]
[-1.5 -8. ]
[-1. -8.]
[-0.5 -8. ]
[ 0. -8.]
[ 0.5 -8. ]
[ 1. -8.]
[ 1.5 -8. ]
[ 2. -8.]
[ 2.5 -8. ]
[ 3. -8.]
[ 3.5 -8. ]
[ 4. -8.]
[ 4.5 -8. ]
[ 5. -8.]
[ 5.5 -8. ]
[ 6. -8.]
[ 6.5 -8. ]
[ 7. -8.]
[ 7.5 -8. ]
[ 8. -8.]
[-8.  -7.5]
[-7.5 -7.5]
[-7.  -7.5]
[-6.5 -7.5]
[-6.  -7.5]
[-5.5 -7.5]
[-5.  -7.5]
[-4.5 -7.5]
[-4.  -7.5]
[-3.5 -7.5]
[-3.  -7.5]
[-2.5 -7.5]
[-2.  -7.5]
[-1.5 -7.5]
[-1.  -7.5]
[-0.5 -7.5]
[ 0.  -7.5]
[ 0.5 -7.5]
[ 1.  -7.5]
[ 1.5 -7.5]
[ 2.  -7.5]
[ 2.5 -7.5]
[ 3.  -7.5]
[ 3.5 -7.5]
[ 4.  -7.5]
[ 4.5 -7.5]
[ 5.  -7.5]
[ 5.5 -7.5]
[ 6.  -7.5]
[ 6.5 -7.5]
[ 7.  -7.5]
[ 7.5 -7.5]
[ 8.  -7.5]
[-8. -7.]
[-7.5 -7. ]
[-7. -7.]
[-6.5 -7. ]
[-6. -7.]
[-5.5 -7. ]
[-5. -7.]
[-4.5 -7. ]
[-4. -7.]
[-3.5 -7. ]
[-3. -7.]
[-2.5 -7. ]
[-2. -7.]
[-1.5 -7. ]
[-1. -7.]
[-0.5 -7. ]
[ 0. -7.]
[ 0.5 -7. ]
[ 1. -7.]
[ 1.5 -7. ]
[ 2. -7.]
[ 2.5 -7. ]


In [32]:
import numpy as np

def generate_scan_pattern():
    x_range = np.arange(-8, 8.1, 0.5)  # Adding 8.1 to include 8
    y_range = np.arange(-8, 8.1, 0.5)
    coordinates = []
    
    for y in y_range:
        if int(y * 2) % 2 == 0:  # Even rows (left to right)
            x_coords = x_range
        else:  # Odd rows (right to left)
            x_coords = x_range[::-1]
            
        for x in x_coords:
            coordinates.append((x, y))
            
    return np.array(coordinates)

# Generate the coordinates
scan_pattern = generate_scan_pattern()

# Print first few coordinates to show the pattern
print("First 10 coordinates:")
for i in range(10):
    print(f"({scan_pattern[i][0]:.1f}, {scan_pattern[i][1]:.1f})")

print("\nShape of pattern:", scan_pattern.shape)

First 10 coordinates:
(-8.0, -8.0)
(-7.5, -8.0)
(-7.0, -8.0)
(-6.5, -8.0)
(-6.0, -8.0)
(-5.5, -8.0)
(-5.0, -8.0)
(-4.5, -8.0)
(-4.0, -8.0)
(-3.5, -8.0)

Shape of pattern: (1089, 2)


In [34]:
17*2

34

In [35]:
34*34

1156

In [40]:
len([i for i in np.arange(-8,8.1,0.5)])

33

In [46]:
33*32

1056

In [42]:
1089-1055-8

26

In [43]:
33*32

1056

In [44]:
16//0.5

32.0

In [45]:
16/0.5

32.0